### 运行方法
- [参考页面](https://qwen.readthedocs.io/zh-cn/latest/training/SFT/llama_factory.html)
1. 进入llamafactor目录
```bash
cd LLaMA-Factory
```
2. 将模型放到相关目录，比如download文件夹，这样你的模型路径就是`download/Qwen2.5-7B-Instruct`

3. 将数据集的两个json文件放到LaMA-Factory项目的data目录。

4. 修改data/dataset_info.json文件，将data_info.json文件的内容粘贴合并进去。

```json
{
  "identity": {
    "file_name": "identity.json"
  },
  ....
  "starcoder_python": {
    "hf_hub_url": "bigcode/starcoderdata",
    "ms_hub_url": "AI-ModelScope/starcoderdata",
    "columns": {
      "prompt": "content"
    },
    "folder": "python"
  },
  "my_dataset": {
    "file_name": "my_dataset.json",
    "formatting": "sharegpt",
    "columns": {
      "messages": "messages"
    },
    "tags": {
      "role_tag": "role",
      "content_tag": "content",
      "user_tag": "user",
      "assistant_tag": "assistant",
      "system_tag": "system"
    }
  }
}
```

6. 输出路径可以为output/Qwen2.5-7B-Instruct
```bash
mkdir -p output/Qwen2.5-7B-Instruct
```

6. 开一个screen命令，挂后台方便。
```bash
# 安装screen
sudo apt install screen
# 开一个screen 后台
screen -S train
```

7. 运行下面的命令开始训练，`cutoff_len`为最大token数，建议设置为2的指数倍，这个参数根据第一步的数据分析得到。
- `my_dataset`即第三步自定义构建的数据集, `>>train.log`将训练过程放到train.log文件。
- `CUDA_VISIBLE_DEVICES=0`用于指定卡0跑模型，如果需要多卡，可以参考官方的教程，加一些并行参数。
- 参数配置可以参考官方给的sft配置文件。[链接](https://github.com/QwenLM/Qwen2.5/blob/main/examples/llama-factory/qwen2-7b-lora-sft.yaml)
- 创建一个bash.sh文件，将运行参数写进去。
```bash
CUDA_VISIBLE_DEVICES=0 python3 src/train.py \
    --stage sft \
    --lora_rank 16 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --lora_target all \
    --per_device_train_batch_size 1 \
    --gradient_accumulation_steps 16 \
    --learning_rate 1.0e-4 \
    --num_train_epochs 1 \
    --lr_scheduler_type cosine \
    --warmup_ratio 0.1 \
    --do_train \
    --use_fast_tokenizer \
    --model_name_or_path download/Qwen2.5-7B-Instruct \
    --dataset my_dataset \
    --template qwen \
    --finetuning_type lora \
    --output_dir output/Qwen2.5-7B-Instruct \
    --overwrite_cache \
    --overwrite_output_dir \
    --ddp_timeout 180000000 \
    --logging_steps 1 \
    --cutoff_len 1024 \
    --save_steps 1000 \
    --plot_loss \
    --bf16 
```
- 执行这个bash文件，运行命令如下：
```bash
./bash.sh > train.log  2>&1
```

### 压缩lora目录文件为zip（可选）
```bash
cd output/Qwen2.5-7B-Instruct
zip -j lora.zip *
```

### 合并lora文件到模型权重

In [ ]:
CUDA_VISIBLE_DEVICES=0 llamafactory-cli export \
    --model_name_or_path download/Qwen2.5-7B-Instruct \
    --adapter_name_or_path output/Qwen2.5-7B-Instruct \
    --template qwen \
    --finetuning_type lora \
    --export_dir output/Qwen2.5-7B-Instruct-Lora-Merge \
    --export_size 2 \
    --export_legacy_format False